In [64]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from datetime import datetime as dt
from datetime import timedelta

In [65]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'keys.json'

credentials = None
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

SAMPLE_SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'

service = build('sheets', 'v4', credentials=credentials)

spread_sheets = {
    'transcations':"transactions!A1:D29001",
    'clients':"clients!A1:C75767",
    'managers':"managers!A1:C14",
    'leads':"leads!A1:F3338"
}

for sheet in spread_sheets:
    result = service.spreadsheets().values().get(
    spreadsheetId=SAMPLE_SPREADSHEET_ID, 
    range=spread_sheets[sheet]).execute()
    
    data = result.get('values', [])
    
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    
    print(spread_sheets[sheet], df.shape)
    
    spread_sheets[sheet] = df
    

transactions!A1:D29001 (29000, 4)
clients!A1:C75767 (75766, 3)
managers!A1:C14 (13, 3)
leads!A1:F3338 (3337, 6)


In [66]:
clients_df = spread_sheets['clients']
leads_df = spread_sheets['leads']
managers_df = spread_sheets['managers']
transactions_df = spread_sheets['transcations']

In [67]:
leads_df['d_utm_source'] = leads_df['d_utm_source'].replace({
    'vk':'vkontakte',
    'insta': 'instagram'
})
leads_df = leads_df[(leads_df.d_utm_source.str.len() > 1) & (leads_df.d_utm_medium.str.len() > 1)]

In [68]:
leads_df['created_at'] = pd.to_datetime(leads_df['created_at'])
leads_df['ts_created_at'] = leads_df[['created_at']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)

C:\Users\Acer\anaconda3\envs\basic_env_p37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Acer\anaconda3\envs\basic_env_p37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
transactions_df['created_at'] = pd.to_datetime(transactions_df['created_at'])
transactions_df['ts_created_at'] = transactions_df[['created_at']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)

In [70]:
leads_df = pd.merge(left=leads_df, right=managers_df, left_on='l_manager_id', right_on='manager_id')
leads_df.head(2)

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,ts_created_at,manager_id,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,1602871689,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2
1,0127f039-0541-11eb-ac18-c412f533dba1,2020-10-03 10:23:52,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,b7b746d1-1adf-11e8-812f-c412f533dba1,1601720632,1e9c5562-0cfc-11ea-abe1-c412f533dba1,manager #8,club #2


In [71]:
leads_df['is_true_lead'] = leads_df['l_client_id'].isin(clients_df['client_id'].unique().tolist())
leads_df['trash_lead'] = ~leads_df['is_true_lead']

In [72]:
temp = leads_df.groupby('l_client_id')['created_at'].min()
leads_df = leads_df.merge(temp, left_on='l_client_id', right_on='l_client_id', how='left')
leads_df['is_first_lead'] = leads_df['created_at_x'] == leads_df['created_at_y']

transactions_df['m_real_amount'] = transactions_df['m_real_amount'].astype(int)
leads_df['full_new_lead'] = ~leads_df['l_client_id'].isin(transactions_df[transactions_df['m_real_amount'] > 0]['l_client_id'].unique().tolist()) & leads_df['is_first_lead'] 

In [73]:
for i in tqdm(range(leads_df.shape[0])):
    lead_row = leads_df.loc[i]
    
    transac = (transactions_df['l_client_id']==lead_row['l_client_id']) & (transactions_df['created_at'] < lead_row['created_at_x']) & (transactions_df['m_real_amount'] > 0)
    leads_df.loc[i, 'has_transactions'] = transac.max()
    
    transac = (transactions_df['l_client_id']==lead_row['l_client_id']) & (transactions_df['created_at']>=lead_row['created_at_x']) & (transactions_df['created_at']<=(lead_row['created_at_x'] + timedelta(days=7)))
    leads_df.loc[i, 'is_7day_buyer'] = transac.max()

leads_df['no_transactions'] = ~leads_df['has_transactions']
leads_df['new_client'] = leads_df['is_first_lead'] & leads_df['no_transactions']
leads_df['is_new_7day_buyer'] = leads_df['new_client'] & leads_df['is_7day_buyer']
    

  0%|          | 0/1984 [00:00<?, ?it/s]

In [74]:
leads_df = leads_df.merge(transactions_df.groupby(['l_client_id'])['m_real_amount'].sum(), left_on='l_client_id', right_on = 'l_client_id')
leads_df['total_amount'] = leads_df['is_new_7day_buyer'] * leads_df['m_real_amount']

In [75]:
leads_df['is_7day_buyer'] = leads_df['is_7day_buyer'].astype('bool')

In [79]:
result_df = leads_df.groupby(['d_utm_medium', 'd_utm_source', 'd_club']).agg(
    count_leads=('lead_id', 'count'),
    count_trash_leads=('trash_lead', 'sum'),
    count_new_leads=('full_new_lead', 'sum'),
    count_buyers=('is_7day_buyer', 'sum'),
    count_new_buyers=('is_new_7day_buyer', 'sum'),
    total_income = ('total_amount', 'sum')
)
result_df
# ['lead_id', 'trash_lead', 'full_new_lead', 'is_7day_buyer', 'is_new_7day_buyer'].count()

count_leads  count_trash_leads  \
d_utm_medium d_utm_source d_club                                    
cpc          google       club #1            5                  0   
                          club #2           12                  0   
                          club #3           19                  0   
                          club #4           13                  0   
             instagram    club #1           34                  0   
                          club #2           62                  0   
                          club #3           48                  0   
                          club #4           58                  0   
             vkontakte    club #1            4                  0   
                          club #2           12                  0   
                          club #3            8                  0   
                          club #4            8                  0   
             yandex       club #1           10                  0   
                          club #2           20                  0   
                          club #3            9                  0   
                          club #4           18                  0   
cpi          sms          club #1            2                  0   
                          club #2            9                  0   
                          club #3            3                  0   
                          club #4            4                  0   
             viber        club #1            9                  0   
                          club #2            5                  0   
                          club #3            3                  0   
                          club #4            9                  0   
organic      vkontakte    club #2            4                  0   
                          club #3            5                  0   
                          club #4            1                  0   
partnerbro   sms          club #2            1                  0   
                          club #4            2                  0   
sms          viber        club #1           11                  0   
                          club #2           16                  0   
                          club #3           10                  0   
                          club #4           13                  0   
socail       vkontakte    club #2            1                  0   
social       ig           club #2            1                  0   
                          club #4            2                  0   
             instagram    club #2            4                  0   
                          club #3            8                  0   
                          club #4            5                  0   
             vkontakte    club #1            2                  0   
                          club #2            7                  0   
                          club #3            5                  0   
                          club #4            8                  0   
             ycard        club #1            1                  0   
                          club #2            2                  0   
utm_medium   utm_source   club #1           18                  0   
                          club #2           54                  0   
                          club #3           19                  0   
                          club #4           39                  0   
             vkontakte    club #1            2                  0   
                          club #2            2                  0   
                          club #4            3                  0   

                                   count_new_leads  count_buyers  \
d_utm_medium d_utm_source d_club                                   
cpc          google       club #1                0             4   
                          club #2                0             4   
                          club #3   

In [80]:
result_df = result_df.reset_index()
result = result_df.values.tolist()
result = [list(result_df.columns)] + result

In [82]:
result_id = '1ZOaVKG0GooCxeZr2EEeCsjuEgmzGbVNjVgU6uSxc9sg'
request = service.spreadsheets().values().update(
    spreadsheetId=result_id, 
    range="final!A1",
    valueInputOption='USER_ENTERED',
    body={
        "values":result
    }
).execute()